# Purpose
Extract data from default TestStand database using default Access NI schema. 
Data is converted to CSVs for import into another tool such as Minitab or analyzed in web app.

One CSV file will be created for each Sequence File name that was called - 
* Numeric Data - data of steps with Numeric Limit Tests; column names step names and tolerances

It is assumed that the entirety of the queried data has the same basic test structure - step names, tolerances,
and step sequences order is not drastically changing. Changes to this will require normalization.

# Process
* Install requirements from requirements.txt (using a virtual environment is highly recommended)
* User is prompted to enter path to database and optional search criteria - data range, user name
* Execute all cells

# Important Notes
* If no pyodbc drivers are listed, then install the correct driver or use the same python bitness as already installed driver

In [4]:
import pyodbc
import pandas as pd

print("---- Microsft Access Drivers ----")
for x in [x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]:
    print(x)

---- Microsft Access Drivers ----
Microsoft Access Driver (*.mdb, *.accdb)


In [5]:
# Get database file from user
from tkinter import *
from tkinter.ttk import *
from tkinter.filedialog import askopenfilename
from tkinter import simpledialog

# we don't want a full GUI, so keep the root window from appearing
Tk().withdraw()

# show an "Open" dialog box and return the path to the selected file
db_filename = askopenfilename(title="Select TestStand Database File to Open", filetypes = (("MS Access","*.mdb"),("MS Access","*.accdb")))

# The DRIVER curly brackets are doubled up to escape themselves
# The DBQ curly brackets are NOT doubled up because they are placeholders
con_string = ('DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};'
              'DBQ={};').format(db_filename)

# Connect to database
cnxn = pyodbc.connect(con_string)
crsr = cnxn.cursor()

In [6]:
# Get all desired test runs filtered by start-date and optional user-name

start_date = simpledialog.askstring('OPTIONAL FILTER', 'Enter start-date filter with format\n #YYYY-MM-DD HH:mm#\n(leave blank to get all dates) ::')
user_name = simpledialog.askstring('OPTIONAL FILTER', 'Enter user-name filter\n(leave blank to get all users) ::')

if start_date == "":
    start_date = '#2022-11-11 00:00#'
    
sql_string = ('SELECT ID, STATION_ID, START_DATE_TIME, EXECUTION_TIME, UUT_SERIAL_NUMBER, UUT_STATUS, UUT_ERROR_MESSAGE ' 
              'FROM UUT_RESULT '
              'WHERE START_DATE_TIME >= {}').format(start_date)

if user_name:
    sql_string += (" AND USER_LOGIN_NAME = '{}'").format(user_name)

crsr.execute(sql_string)
rows = crsr.fetchall()

In [7]:
# For each seqeunce file, generate empty tables. 
# NOTE: Step_SeqCall does not have start-date-time so there will be minor inefficiency

from pathlib import Path

sql_string2 = """
SELECT STEP_SEQCALL.STEP_RESULT, STEP_SEQCALL.SEQUENCE_FILE_PATH 
FROM STEP_SEQCALL
"""

crsr.execute(sql_string2)
seqs = crsr.fetchall()

# Declaration of empty dictionary for dynamic output tables
tbl_dict = {}

# Declaration of empty sequence list
seq_list = []

# Create look-up table mapping x = Step_Parent to y = Sequence_Name as Tuple (x,y)
for seq in seqs:
    
    # Update raw filepaths to final table names
    seq[1] = 'Test Data ' + Path(seq[1]).stem
    
    # Create list containing each unique sequence in the .mdb file
    while seq[1] not in seq_list:
        seq_list.append(seq[1])
    
    # Assign list entries as keys with empty array as each entry's value (will populate in next section)
    for seq_name in seq_list:
        tbl_dict[f'{seq_name}'] = []

In [8]:
# For each test run, generate tables
for row in rows:
    
    # Overall Test Metadata
    data = {
        'Test Start': row.START_DATE_TIME, 'Serial Number': row.UUT_SERIAL_NUMBER, 
        'Test ID': row.ID, 'Station ID': row.STATION_ID, 'Test Status': row.UUT_STATUS
    }
    
    # Step Data
    crsr.execute(f"""
    SELECT  STEP_RESULT.ID, 
            STEP_RESULT.STEP_PARENT, 
            STEP_RESULT.STEP_NAME, 
            STEP_RESULT.STEP_TYPE, 
            STEP_RESULT.STATUS, 
            STEP_RESULT.ORDER_NUMBER,
            PROP_RESULT.ID AS PROP_ID, 
            PROP_RESULT.TYPE_NAME, 
            PROP_RESULT.DATA, 
            PROP_RESULT.NAME,
            PROP_NUMERICLIMIT.COMP_OPERATOR AS COP, 
            PROP_NUMERICLIMIT.HIGH_LIMIT AS HL, 
            PROP_NUMERICLIMIT.LOW_LIMIT AS LL, 
            PROP_NUMERICLIMIT.UNITS AS UNITS
    FROM (STEP_RESULT LEFT JOIN PROP_RESULT ON STEP_RESULT.ID = PROP_RESULT.STEP_RESULT)
         LEFT JOIN PROP_NUMERICLIMIT ON PROP_RESULT.ID = PROP_NUMERICLIMIT.PROP_RESULT
    WHERE STEP_RESULT.UUT_RESULT = {row.ID}
    ORDER BY STEP_RESULT.ORDER_NUMBER ASC
    """)
    
    step_rows = crsr.fetchall()
    
    for step_row in step_rows:
        
        # Convert data to Python types
        val = None
        if step_row.TYPE_NAME == "Boolean":
            val = bool(step_row.DATA)

        elif step_row.TYPE_NAME == "Number" or step_row.TYPE_NAME == "NumericLimitTest":
            val = float(step_row.DATA)

        # Extract desired data
        if (
                 step_row.STEP_TYPE == "NumericLimitTest" and 
                 step_row.STATUS != "Skipped" and
                 val is not None
            ):
            # Construct limit info string based on comparison type
            limit_info = ""
            if step_row.COP == "LT":
                #limit_info = f"< {step_row.LL:.4F}"
                limit_info = f"< {step_row.LL}"
            elif step_row.COP == "LE":
                limit_info = f"<= {step_row.LL}"
            elif step_row.COP == "GT":
                limit_info = f"> {step_row.LL}"
            elif step_row.COP == "GE":
                limit_info = f">= {step_row.LL}"
            elif step_row.COP == "GTLT":
                limit_info = f"{step_row.LL} < x < {step_row.HL}"
            elif step_row.COP == "GELE":
                limit_info = f"{step_row.LL} <= x <= {step_row.HL}"
            elif step_row.COP == "EQT":
                limit_info = f"{step_row.LL} <= x <= {step_row.HL}"
            elif step_row.COP == "EQ":
                limit_info = f" == {step_row.LL}"
            else:
                break

            # When looping, different step runs will have repeated step name.
            # Need to append next numeric suffix to separate runs
            repeat_index = 0
            key_name = f"{step_row.STEP_NAME} ({step_row.UNITS}) {limit_info} [{repeat_index}]"
            
            while key_name in data:
                repeat_index += 1
                key_name = f"{step_row.STEP_NAME} ({step_row.UNITS}) {limit_info} [{repeat_index}]"

            # Finally insert data value only keeping up to 3 decimal places
            data[key_name] = round(val,3)
            
    # Convert STEP_PARENT index to the corresponding sequence file dictionary value
    dict_val = next((y for x, y in seqs if x == step_row.STEP_PARENT), None)
    
    # Append data to the appropriate output table according to dictionary value
    tbl_dict[f'{dict_val}'].append(data)
    
print("---- Test Data generated ----")

---- Test Data generated ----


In [9]:
# Write out tables to CSVs

for seq_name in seq_list:
    pd.DataFrame.from_records(tbl_dict[f'{seq_name}']).to_csv(seq_name +'.csv', index=False)
    print(f"---- {seq_name} exported ----")

---- Test Data OMS exported ----
---- Test Data VDM exported ----
---- Test Data PMU exported ----
